# <b>General validation</b>

<b>Required libaries:</b>

In [ ]:
!pip install ultralytics
!pip install onnx
!pip install onnxruntime

<b>Necessary imports:</b>

In [ ]:
from ultralytics import YOLO
from google.colab import drive
import requests
from PIL import Image, ImageOps
from io import BytesIO
import os
import time
import numpy as np

In [ ]:
# establish connection to google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Prediction of the different models on a specific image

In [ ]:
# function to load models
def load_models(model_paths):
    print("Loading models...")
    models = []
    for model_path in model_paths:
        try:
            model = YOLO(model_path)
            models.append((model, model_path))
            print(f"Loaded model: {model_path}")
        except Exception as e:
            print(f"Failed to load model {model_path}: {e}")
    return models

# function to get image
def load_image_from_url(url):
    print(f"Downloading image from {url}...")
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

# function to run prediction
def run_predictions(models, image):
    results = {}
    for i, (model, model_path) in enumerate(models):
        start_time = time.time()
        results[model_path] = model.predict(image)
        end_time = time.time()
        print(f"Model {os.path.basename(model_path)} prediction time: {end_time - start_time:.2f} seconds.")
    return results

# function to save results
def save_results(results, output_dir="drive/MyDrive/PA2/predictions"):
    for model_path, result in results.items():
        model_name = os.path.splitext(os.path.basename(model_path))[0]
        img_with_boxes = Image.fromarray(result[0].plot())
        output_path = os.path.join(output_dir, f"{model_name}.png")
        img_with_boxes.save(output_path)
        print(f"Saved {model_name} result to {output_path}")



In [ ]:
# define model paths
model_paths = [
    'drive/MyDrive/PA2/models/yolov8x.onnx',  # standard yolov8x
    'drive/MyDrive/PA2/models/yolov8x_quantized.onnx'   # quantized yolov8x
]


In [ ]:
# load models
models = load_models(model_paths)

# load image
image_url = 'https://farm9.staticflickr.com/8097/8427292166_9cb677b6fa_z.jpg'
image = load_image_from_url(image_url)


Loading models...
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loaded model: drive/MyDrive/PA2/models/yolov8x.onnx
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loaded model: drive/MyDrive/PA2/models/yolov8x_quantized.onnx


In [ ]:
# Run predictions for each model
results = run_predictions(models, image)
# Display or process results
save_results(results)

Loading drive/MyDrive/PA2/models/yolov8x.onnx for ONNX Runtime inference...

0: 640x640 1 person, 1 bicycle, 1 dog, 1 backpack, 9529.6ms
Speed: 2.8ms preprocess, 9529.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Model yolov8x.onnx prediction time: 11.68 seconds.
Loading drive/MyDrive/PA2/models/yolov8x_quantized.onnx for ONNX Runtime inference...

0: 640x640 1 person, 1 bicycle, 1 dog, 1 backpack, 2140.5ms
Speed: 2.6ms preprocess, 2140.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Model yolov8x_quantized.onnx prediction time: 2.63 seconds.
Saved yolov8x result to drive/MyDrive/PA2/predictions/yolov8x.png
Saved yolov8x_quantized result to drive/MyDrive/PA2/predictions/yolov8x_quantized.png


### Calculate the compression rate

In [ ]:
# function to get file size
def get_file_size_mb(path):
    size_in_bytes = os.path.getsize(path)
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

# map model paths to model names
models = {
    'yolov8x': model_paths[0],
    'yolov8x_quantized': model_paths[1]
}

# get size of original and quantized model
yolov8x_size = get_file_size_mb(models['yolov8x'])
yolov8x_quantized_size = get_file_size_mb(models['yolov8x_quantized'])

# calculate compression rate
yolov8x_compression_rate = yolov8x_size / yolov8x_quantized_size

# print the results
print(f"YOLOv8x model size: {yolov8x_size:.2f} MB")
print(f"YOLOv8x quantized model size: {yolov8x_quantized_size:.2f} MB")
print(f"YOLOv8x compression rate: {yolov8x_compression_rate:.2f}")


YOLOv8x model size: 260.40 MB
YOLOv8x quantized model size: 66.04 MB
YOLOv8x compression rate: 3.94
